<h1>TDS Analysis</h1>

<h2>Select file</h2>

In [2]:
import numpy as np
import igor.igorpy as igor
import os
from os import listdir
from os.path import isfile, join
import sys
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import clear_output
import math
import struct
from importlib import reload
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame as df

sys.path.append(os.getcwd() + '/Tools/')
import TDSAnalysis
reload(TDSAnalysis)
import AnalysisTools
reload(AnalysisTools)
dt = AnalysisTools.DataTools()

ParameterFolder = os.getcwd()+'/Parameters'
FitsFolder = os.getcwd()+'/Fits'

##### Widgets #####

ParameterFile = widgets.Dropdown(
    options=dt.FileList(ParameterFolder,['.yaml','TPD']),
    description='Select File',
    layout=Layout(width='70%'),
    style = {'description_width': '150px'},
    disabled=False,
)

display(ParameterFile)

##### Functions #####

def FileList() :
    return [f for f in listdir(FolderPath.value) if isfile(join(FolderPath.value, f))]

Dropdown(description='Select File', layout=Layout(width='70%'), options=('TPD181019_03 - CH3OH_O2_Ni', 'TPD181…

<h2>TDS traces</h2>

In [3]:
##### Parameters #####

MinTemperature = 115
MaxTemperature = 2000

##### Load Data #####

reload(AnalysisTools)
dt = AnalysisTools.DataTools()

reload(TDSAnalysis)
tds = TDSAnalysis.TDS((ParameterFolder,ParameterFile.value))
Data = tds.Data
Data = dt.TrimData(Data,MinTemperature,MaxTemperature)
Masses = tds.Parameters['Masses']

##### Functions #####

def SimulateTrace_Clicked(b) :
    with out :
        clear_output(True)
        
        reload(TDSAnalysis)
        tds = TDSAnalysis.TDS((ParameterFolder,ParameterFile.value))
        tds.SimulateData(HeatingRate.value)
        SimulatedData = dt.TrimData(tds.SimulatedData,MinTemperature,MaxTemperature)

        fig = go.Figure()
        for Trace in Data :
            if Trace != 'Time (s)' :
                fig.add_trace(go.Scatter(x=Data.index,y=Data[Trace],name=Trace,mode='lines'))
        for Trace in SimulatedData :
            fig.add_trace(go.Scatter(x=SimulatedData.index,y=SimulatedData[Trace],name=Trace,mode='lines'))
        fig.update_layout(xaxis_title='Temperature (K)',yaxis_title='Fit Value',title=tds.Parameters['Description'],legend_title='')
        fig.show()

        fig = px.line(tds.SimulatedCoverages)
        fig.update_layout(yaxis_title='Coverage',showlegend=False,height=100)
        fig.show()
        display(Save2File)

def Save2File_Clicked(b) :
    FitsFile = FitsFolder +'/' + ParameterFile.value + '.hdf'
    Data.to_hdf(FitsFile,'Data',mode='w')
    tds.Assignments.to_hdf(FitsFile,'Assignments',mode='a')
Save2File = widgets.Button(description="Save to File")
Save2File.on_click(Save2File_Clicked)

##### Widgets #####

HeatingRate = widgets.FloatText(
    value=np.around(tds.Parameters['HeatingRate'],3),
    description='Heating Rate (K/s):',
    layout=Layout(width='25%'),
    style = {'description_width': '140px'},
    disabled=False
)

SimulateTrace = widgets.Button(description="Simulate Traces")
SimulateTrace.on_click(SimulateTrace_Clicked)

display(widgets.Box([SimulateTrace,HeatingRate]))

out = widgets.Output()

##### Plot Data #####

with out :
    fig = go.Figure()
    for Trace in Data :
        if Trace != 'Time (s)' :
            fig.add_trace(go.Scatter(x=Data.index,y=Data[Trace],name=Trace,mode='lines'))
    fig.update_layout(xaxis_title='Temperature (K)',yaxis_title='Fit Value',title=tds.Parameters['Description'],legend_title='')
    fig.show()
    display(Save2File)

out

Box(children=(Button(description='Simulate Traces', style=ButtonStyle()), FloatText(value=0.734, description='…

Output()

<h2>Numerically integrate traces</h2>

In [4]:

#Make indexes nice to work with
def indexTrim(Data) :
    boo = np.zeros(len(Data), int)
    for a, b in enumerate(boo) :
        boo[a] = a
    Data = Data.set_index(boo)
    return Data

#picking a range
def DataRange(d, lower, upper) :
    td = d.iloc[lower:upper]
    return td

#find mean of points in a dataframe for a given mass
def Baseline(d, mass) :
    base = np.mean(d[mass])
    return base

def integrate(d, base, mass) :
    #get deltaT by just taking the difference between first two temp values

    integrateSum = 0
    first = 0
    
    while first < len(d)-1 :
        deltaT = d.index.values[1] -d.index.values[0]
        #print(first)
        second =  first + 1

        y1 = float(d[mass].values[first])
        y2 = float(d[mass].values[second])
        deltaY = y2-y1
  
        rectA = deltaT * (y1-base)
        triA = deltaT * (deltaY)/2

        integrateSum = integrateSum + rectA + triA
        first = first + 1
        
    return integrateSum

d = indexTrim(Data)
baserange = DataRange(d, 50, 100)
base = Baseline(baserange, 'Mass 28')
print('Base is :', base)
print('Base integration check is :', integrate(baserange, base, 'Mass 28'))
print('integrated value is :', integrate(d, base, 'Mass 28'))

Base is : 14688.0
Base integration check is : -712.0
integrated value is : 3095768.0


<h2>File info</h2>

This script analyzes TPD data. 

The following external packages must be installed:
- lmfit
- igor
- plotly
- xarray

-----------------------
Created by Jerry LaRue, larue@chapman.edu, 12/2018

Last modified by Jerry LaRue, larue@chapman.edu, 01/2021